In [14]:
import sys
import numpy as np
from typing import List

We consider H_2 molecule first
There is a 1s orbital per hydrogen atom
Each of them is given by a linear combination of basis functions, here Gaussians.

In [18]:
class primitive_gaussian():

    def __init__(self, 
                 alpha: float,
                 coeff: float,
                 coordinates: List,
                 l1: int = 0,
                 l2: int = 0,
                 l3: int = 0,
                 ) -> None:
        """ Constructor function for the primitive_gaussian class.

        l1, l2, l3 are equal to 0 for an s orbital. (focus on H_2 molecule first).
        """

        self.alpha = alpha
        self.coeff = coeff
        self.coordinates = np.array(coordinates)
        self.l1, self.l2, self.l3 = l1, l2, l3
        self.A = (2.0 * alpha / np.pi)**0.75  # normalization factor, + other temrs for l1, l2, l3 > 0

In [25]:
def overlap(molecule: List) -> None:
    nbasis = len(molecule)

    S = np.zeros([nbasis, nbasis])

    for i in range(nbasis):
        for j in range(nbasis):
            nprimitives_i = len(molecule[i])
            nprimitives_j = len(molecule[j])

            for k in range(nprimitives_i):
                for l in range(nprimitives_j):
                    
                    # normalization constant of the total overlap
                    N = molecule[i][k].A * molecule[j][l].A
                    p = molecule[i][k].alpha + molecule[j][l].alpha
                    q = molecule[i][k].alpha * molecule[j][l].alpha / p
                    Q = molecule[i][k].coordinates - molecule[j][l].coordinates
                    Q2 = np.dot(Q, Q)

                    S[i, j] += N * molecule[i][k].coeff * molecule[j][l].coeff * np.exp(-q * Q2) * (np.pi / p) ** (3/2) 

    return S


In [26]:
# STO-3G basis for 1s orbital on hydrogen
H1_pg1a = primitive_gaussian(alpha=0.3425250914E+01, 
                             coeff=0.1543289673E+00, 
                             coordinates=[0, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H1_pg1b = primitive_gaussian(alpha=0.6239137298E+00, 
                             coeff=0.5353281423E+00, 
                             coordinates=[0, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H1_pg1c = primitive_gaussian(alpha=0.1688554040E+00, 
                             coeff=0.4446345422E+00, 
                             coordinates=[0, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H2_pg1a = primitive_gaussian(alpha=0.3425250914E+01, 
                             coeff=0.1543289673E+00, 
                             coordinates=[1.2, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H2_pg1b = primitive_gaussian(alpha=0.6239137298E+00, 
                             coeff=0.5353281423E+00, 
                             coordinates=[1.2, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H2_pg1c = primitive_gaussian(alpha=0.1688554040E+00, 
                             coeff=0.4446345422E+00, 
                             coordinates=[1.2, 0, 0],
                             l1=0,
                             l2=0,
                             l3=0)
H1_1s = [H1_pg1a, H1_pg1b, H1_pg1c] # 1s orbital on the first hydrogen atom
H2_1s = [H2_pg1a, H2_pg1b, H2_pg1c] # 1s orbital on the second hydrogen atom
molecule = [H1_1s, H2_1s]
overlap(molecule)




# 6-31G basis for 1s and 2s orbitals on hydrogen



molecule = [H1_1s, H1_2s, H2_1s, H2_2s]


array([[1.        , 0.72864938],
       [0.72864938, 1.        ]])